In [13]:
#setup libs

%pip install yfinance

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [14]:
# import libs

import json
import os
from datetime import datetime

import yfinance as yf

In [15]:
# agent Yahoo Finance
def fetch_stock_price(ticket):
  stock = yf.download(ticket, start='2023-08-08', end='2024-08-20')
  return stock

<function fetch_stock_price at 0x7f30c0068700>
